# DREAM data reduction

We begin with relevant imports.
We will be using tutorial data downloaded with `pooch`.
If you get an error about a missing module `pooch`, you can install it with `!pip install pooch`:

In [ ]:
import scipp as sc

from ess import dream, powder
import ess.dream.data  # noqa: F401
from ess.powder.types import *

## Create and configure the workflow

We begin by creating the Dream (Geant4) workflow object which is a skeleton for reducing Dream data, with pre-configured steps.

In [ ]:
workflow = dream.DreamGeant4Workflow(run_norm=powder.RunNormalization.proton_charge)

We then need to set the missing parameters which are specific to each experiment
(the keys are types defined in [essdiffraction.powder.types](../generated/modules/ess.diffraction.powder.types.rst)):

In [ ]:
workflow[Filename[SampleRun]] = dream.data.simulated_diamond_sample()
workflow[Filename[VanadiumRun]] = dream.data.simulated_vanadium_sample()
workflow[Filename[BackgroundRun]] = dream.data.simulated_empty_can()
workflow[CalibrationFilename] = None
workflow[NeXusDetectorName] = "mantle"
# The upper bounds mode is not yet implemented.
workflow[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.drop
# Edges for binning in d-spacing
workflow[DspacingBins] = sc.linspace("dspacing", 0.0, 2.3434, 201, unit="angstrom")
# Mask in time-of-flight to crop to valid range
workflow[TofMask] = lambda x: (x < sc.scalar(0.0, unit="ns")) | (
        x > sc.scalar(86e6, unit="ns")
)
workflow[TwoThetaMask] = None
workflow[WavelengthMask] = None
# No pixel masks
workflow = powder.with_pixel_mask_filenames(workflow, [])

## Use the workflow

We can visualize the graph for computing the final normalized result for intensity as a function of time-of-flight:

In [ ]:
workflow.visualize([IofTof, ReducedTofCIF], graph_attr={"rankdir": "LR"})

We then call `compute()` to compute the result:
(The `cif` object will later be used to write the result to disk.)

In [ ]:
results = workflow.compute([IofTof, ReducedTofCIF])
result = results[IofTof]
cif_data = results[ReducedTofCIF]

In [ ]:
histogram = result.hist()
histogram.plot()

We can now save the result to disk:

In [ ]:
cif_data.comment = """This file was generated with the DREAM data reduction user guide
in the documentation of ESSdiffraction.
See https://scipp.github.io/essdiffraction/
"""
cif_data.save('reduced.cif')

## Compute intermediate results

For inspection and debugging purposes, we can also compute intermediate results.
To avoid repeated computation (including costly loading of files), we can request multiple results at once, including the final result, if desired.
For example:

In [ ]:
intermediates = workflow.compute(
    (
        DataWithScatteringCoordinates[SampleRun],
        MaskedData[SampleRun],
    )
)

intermediates[DataWithScatteringCoordinates[SampleRun]]

In [ ]:
two_theta = sc.linspace("two_theta", 0.8, 2.4, 301, unit="rad")
intermediates[MaskedData[SampleRun]].hist(two_theta=two_theta, wavelength=300).plot(
    norm="log"
)

## Grouping by scattering angle

The above workflow focuses the data by merging all instrument pixels to produce a 1d d-spacing curve.
If instead we want to group into $2\theta$ bins, we can alter the workflow parameters by adding some binning in $2\theta$:

In [ ]:
workflow[TwoThetaBins] = sc.linspace(
    dim="two_theta", unit="rad", start=0.8, stop=2.4, num=17
)

In [ ]:
grouped_dspacing = workflow.compute(IofDspacingTwoTheta)
grouped_dspacing

In [ ]:
angle = sc.midpoints(grouped_dspacing.coords["two_theta"])
sc.plot(
    {
        f"{angle[group].value:.3f} {angle[group].unit}": grouped_dspacing[
            "two_theta", group
        ].hist()
        for group in range(2, 6)
    }
)

In [ ]:
grouped_dspacing.hist().plot(norm="log")

##  Normalizing by monitor

The workflow used above normalizes the detected counts by proton charge.
Alternatively, ESSdiffraction can normalize by a monitor.
In this example, this is DREAM's cave monitor.

There are two options for normalizing by monitor:
1. Normalize by a wavelength-histogram of the monitor counts ([normalize_by_monitor_histogram](../../generated/modules/ess.powder.correction.normalize_by_monitor_histogram.rst)).
2. Normalized by the integral over all wavelength bins ([normalize_by_monitor_integrated](../../generated/modules/ess.powder.correction.normalize_by_monitor_integrated.rst)).

Here, we use option 1.
Option 2 can be chosen by constructing a workflow using `run_norm=powder.RunNormalization.monitor_integrated`.

Construct a workflow as before but select normalization by monitor histogram:

In [ ]:
workflow = dream.DreamGeant4Workflow(run_norm=powder.RunNormalization.monitor_histogram)

In addition to the parameters used before, we also need to provide filenames for monitor data and a position of the monitor as that is not saved in the simulation files:

In [ ]:
workflow[MonitorFilename[SampleRun]] = dream.data.simulated_monitor_diamond_sample()
workflow[MonitorFilename[VanadiumRun]] = dream.data.simulated_monitor_vanadium_sample()
workflow[MonitorFilename[BackgroundRun]] = dream.data.simulated_monitor_empty_can()
workflow[CaveMonitorPosition] = sc.vector([0.0, 0.0, -4220.0], unit='mm')

# These are the same as at the top of the notebook:
workflow[Filename[SampleRun]] = dream.data.simulated_diamond_sample()
workflow[Filename[VanadiumRun]] = dream.data.simulated_vanadium_sample()
workflow[Filename[BackgroundRun]] = dream.data.simulated_empty_can()
workflow[CalibrationFilename] = None
workflow[NeXusDetectorName] = "mantle"
workflow[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.drop
workflow[DspacingBins] = sc.linspace("dspacing", 0.0, 2.3434, 201, unit="angstrom")
workflow[TofMask] = lambda x: (x < sc.scalar(0.0, unit="ns")) | (
        x > sc.scalar(86e6, unit="ns")
)
workflow[TwoThetaMask] = None
workflow[WavelengthMask] = None
workflow = powder.with_pixel_mask_filenames(workflow, [])

In [ ]:
workflow.visualize(IofTof, graph_attr={"rankdir": "LR"})

In [ ]:
results = workflow.compute((IofTof, WavelengthMonitor[SampleRun, CaveMonitor]))
normalized_by_monitor = results[IofTof]
monitor = results[WavelengthMonitor[SampleRun, CaveMonitor]]
monitor

In [ ]:
monitor.plot()

Comparing the final, normalized result shows that it agrees with the data that was normalized by proton-charge:

In [ ]:
sc.plot({
    'By proton charge': histogram,
    'By monitor': normalized_by_monitor.hist()
})